In [ ]:
! pip install scikit-learn

In [ ]:
import finpy_tse as fpy
import pandas as pd
import sqlite3 as db
import pytse_client as tse
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

portfollio = 'افران توان اوج پالایش وخارزم اعتلا وبوعلی تمشک گوهران وهامون ومدیر وتوسم وسبحان بوعلی وسکاب مدیریت وصنعت کیان وسپه ومهان'
portfollio_eng = 'vmahan vsepah kian vsanaat modiriat vsakab booali vsobhan vtosam vmodir vhamoon goharan tameshk vbooali alaa vkharazm palayesh owj tavaan afran'
my_pf = portfollio.split()
my_pf_eng = portfollio_eng.split()
my_pf_eng = my_pf_eng[::-1]



## دریافت ریز معاملات

In [ ]:
i = 1
print(my_pf_eng[i])
symbol = my_pf[i]
eng = my_pf_eng[i]
symbol

In [ ]:
symbol = my_pf[i]
eng_symbol = my_pf_eng[i]
df_price_details =fpy.Get_IntradayTrades_History(
                    stock=symbol,
                    start_date='1402-04-13',
                    end_date='1402-04-14',
                    jalali_date=True,
                    combined_datatime=False,
                    show_progress=True)
df_price_details.head()

اضافه کردن ریز معاملات به دیتا بیس

In [ ]:
conn = db.connect('my_bours_price_details.db')
cur = conn.cursor()

df_price_details.reset_index(inplace=True)
df_price_details_new = df_price_details.rename(columns={"J-Date": "Date"})    
df_price_details_new.to_sql(eng_symbol,conn,if_exists='append',index=False)

conn.commit()
cur.close()
conn.close()

## دریافت عمق فقط برای یک نماد

In [ ]:
startDate = '1402-04-10'
endDate = '1402-04-14'
stk = symbol

conn = db.connect('my_bours_omgh.db')
cur = conn.cursor()
df_omgh = fpy.Get_IntradayOB_History(
            stock= stk ,
            start_date= startDate,
            end_date= endDate,
            jalali_date=True,
            combined_datatime=False,
            show_progress=True)
print('done')

df_omgh.reset_index(inplace=True)
df_omgh = df_omgh.rename(columns={"J-Date": "Date"})
df_omgh.to_sql(f'{eng}_omgh_daily',conn,if_exists='append',index=False)

conn.commit()
cur.close()
conn.close()

## دریافت عمق فقط برای چند نماد

In [ ]:
startDate = '1402-01-28'
endDate = '1402-01-30'

conn = db.connect('my_bours_omgh.db')
cur = conn.cursor()


for sym,eng in zip(my_pf,my_pf_eng):
    df_omgh = fpy.Get_IntradayOB_History(
            stock=sym,
            start_date= startDate,
            end_date= endDate,
            jalali_date=True,
            combined_datatime=False,
            show_progress=True)
    
    df_omgh.reset_index(inplace=True)
    df_omgh_new = df_omgh.rename(columns={"J-Date": "Date"})    
    df_omgh_new.to_sql(f'{eng}_omgh_daily',conn,if_exists='append',index=False)
    print(f'{sym} is done')


# close the connection object
conn.commit()
cur.close()
conn.close()

# دریافت قیمت روزانه 

In [ ]:
conn = db.connect('daily_price.db')
cur = conn.cursor()

for sym,eng_sym in zip(my_pf,my_pf_eng):
    ticker = tse.Ticker(sym)
    res = tse.download(symbols=sym, write_to_csv=False)
    df = res[sym]
    df.to_sql(f'{eng_sym}_daily',conn,if_exists='append',index=False)
    print(f'{sym} is done')

conn.commit()
cur.close()
conn.close()

#  بارگزاری دیتاهای پرتفو از دیتا بیس

In [ ]:
conn = db.connect('daily_price.db')
chart_data = {}

for com in my_pf_eng:
    com_value = pd.read_sql(f"SELECT * FROM {com}_daily", conn)
    chart_data.update({com : com_value})


conn.close()
chart_data

In [ ]:
x = new_df.index
y = new_df['kian']
y2 = new_df['vsakab']

fig, ax = plt.subplots()
ax.plot(x, y)
ax.plot(x, y2)

plt.show()


In [ ]:
df1 = pd.DataFrame({'date': ['2022-01-01', '2022-01-02', '2022-01-03'], 'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'date': ['2022-01-01', '2022-01-03'], 'A': [7, 8], 'B': [9, 10]})
df3 = pd.DataFrame({'date': ['2022-01-02'], 'A': [11], 'B': [12]})

new_df = pd.concat([df1.set_index('date')['A'].rename('afran'),
                    df2.set_index('date')['A'].rename('tavaan'), df3.set_index('date')['A'].rename('kian')], axis=1)

print(new_df)

# بارگزاری چند تیبل از یک دیتابیس

In [ ]:
conn = db.connect('daily_price.db')

# Execute a SQL query and store the results in a Pandas dataframe
df_tables = pd.read_sql('SELECT * FROM table1 JOIN table2 ON table1.id = table2.id JOIN table3 ON table1.id = table3.id JOIN table4 ON table1.id = table4.id', conn)

# Close the database connection
conn.close()

# بارگزاری چند ستون از یک تیبل

In [ ]:
conn = db.connect('daily_price.db')

# Execute a SQL query and store the results in a Pandas dataframe
df_tavan = pd.read_sql('SELECT date, adjClose, volume, count FROM توان_daily_price', conn)

# Close the database connection
conn.close()
df_tavan

In [ ]:
>>> from sklearn.preprocessing import Normalizer
>>> X = [[4, 1, 2, 2],
...      [1, 3, 9, 3],
...      [5, 7, 5, 1]]
>>> transformer = Normalizer().fit(X)  # fit does nothing.
>>> transformer

In [ ]:
transformer.transform(X)